# Look at Outcomes from Ward with Covariates

Use the multi-state model to look at outcomes by age.


In [ ]:
#### Import required packages
%matplotlib inline
import numpy as np
import scipy as sp
import scipy.stats as st
import scipy.special as sp
import scipy.optimize as op
import xarray as xr
import pandas as pd
from scipy import integrate
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import networkx as nx
import seaborn as sns
import pymc3 as pm
import random
import arviz as az
from datetime import datetime, date
from datetime import timedelta
import json

In [ ]:
#### Define input variables. All variable should hopefully be here, so after adjusting these each week,
#### the code can hopefully be run without making further adjustments.

###### What date did we extract the data?
file_date = 'simulated_30May' # Update to the date format used for the input files
dtpull = date(2020,5,27) # update to extract date, best to use 2 days prior to current date
cur_date = dtpull # Tell the code that the current date is the date of the last included admission, rather than today.
forecast_start = date(2020,5,30) # Enter the date we want forecasts to start, for GM aggregate use Tuesday of each week.
######

###### Growth rate and initial conditions for daily admissions trend
admissions_conditions = pd.read_csv('admissions_conditions_'+file_date+'.csv') # load admissions trends
I0_0 = admissions_conditions.initial_value.values[0] # current expected number of admissions: age group 0
rate_0 = admissions_conditions.out_mean.values[0] # growth rate of admissions: age group 0 
I0_1 = admissions_conditions.initial_value.values[1] # current expected number of admissions: age group 1
rate_1 = admissions_conditions.out_mean.values[1] # growth rate of admissions: age group 1
I0_2 = admissions_conditions.initial_value.values[2] # current expected number of admissions: age group 2
rate_2 = admissions_conditions.out_mean.values[2] # growth rate of admissions: age group 2
I0_3 = admissions_conditions.initial_value.values[3] # current expected number of admissions: age group 3
rate_3 = admissions_conditions.out_mean.values[3] # growth rate of admissions: age group 3 
rate_overall = admissions_conditions.out_mean.values[4] # growth rate of admissions across all age groups
######

###### Approximating age groups using the all age group growth rate?
###### This is useful is we have insufficient data to fit growth rates for some of the age groups
approx_growth = False # Set to true if this is required
######

###### Which age groups go to ICU?
ICU_0 = True # any 0-25 patients go to ICU?
ICU_1 = True # any 26-50 patients go to ICU?
ICU_2 = True # any 51-75 patients go to ICU?
ICU_3 = True # any 76+ patients go to ICU?
######

###### How many samples/simulation should we run? As high as possible is best, but increases computational time
nboot = 100 # specify number of sample to draw from MCMC posterior. The more samples used gives a better representation
npat = 1000 # specify number of patients to simulate for posterior credibility
######

###### How many MCMC samples should be test? Again, as high as possible is best
draw_num = 1500 # number of iterations to keep
tune_num = 2500 # number of run in iterations to burn
######

###### How many independent chains should we run and how many cores can we use?
chain_num = 1 # number of independent chains to run
core_num  = 4 # number of cores to use, must be less than or equal to chain_num
######

###### Do we have saved MCMC traces? This enables the forecast to be rerun without fitting the data each week.
## This can be useful as the fitted distributions are unlikely to change much each week.
saved = 0 # load saved trace into xhats variable instead of running a new sample
######

###### Do we want to use combination of Burr and Weibull or just Weibull. If sample size is very small, just Weibull might be best
global_dist = 0 # If "1" use a single distribution for all transitions, otherwise use combination of Burr and Weibull
######

generate_los_estimates = 1




In [ ]:
'Load data - start'

In [ ]:
### Load length of stay data and census data

df = pd.read_csv('Current_COVID_'+file_date+'.csv')

census_covariates = pd.read_csv('census_covariates_'+file_date+'.csv').values


In [ ]:
'Load data - end'

In [ ]:
'Define states and transitions - start'

In [ ]:
state_names = (
    "Acute Ward",
    "ICU",
    "Recovery Ward",
    "Discharge",
    "Death",
)
nstates = len(state_names)

allowed_transitions = (
    ("Acute Ward", "ICU"),
    ("Acute Ward", "Discharge"),
    ("Acute Ward", "Death"),
    ("ICU", "Recovery Ward"),
    ("ICU", "Death"),
    ("Recovery Ward", "Discharge"),
)
ntrans = len(allowed_transitions)

state_dict = {state: loc+1 for loc, state in enumerate(state_names)}
state_name_dict = {loc+1: state for loc, state in enumerate(state_names)}
IJ = [(state_dict[xy[0]],state_dict[xy[1]]) for xy in allowed_transitions]
DG = nx.DiGraph()
DG.add_edges_from(IJ)
plt.figure(figsize=(5,5))
pos=nx.circular_layout(DG)
nx.draw(DG,pos)
nx.draw_networkx_labels(DG,pos,labels=state_name_dict)
plt.show()

Qbase = nx.to_numpy_array(DG,dtype='int',nodelist=range(1,nstates+1))
Qbase # Matrix of allowed transitions: G[i,j] is from i to j

nxy = np.shape(Qbase)
TransNum = Qbase.copy()
k=1
for i in range(0,nxy[0]):
    for j in range(0,nxy[1]):
        if (Qbase[i,j] > 0):
            TransNum[i,j] = k
            k +=1
TransNum

In [ ]:
'Define states and transitions - end'

In [ ]:
'Define required functions - start'

In [ ]:
### Return a Weibull scale parameter from the linear predictor
def get_scale(x, k=1.0, ATF=True):
    if ATF:
        return np.exp(-x)
    else:
        return np.exp(-x/k)

In [ ]:
#### Define function for simulating indicative scenarios over the next few months
def indicative_simulator(scenario,nboot): # scenario is 1 (exponential growth) or 0 (constant admissions)
    #### Define age indicator variable and add the variable group names
    aci = (
        (0),
        (1),
        (2),
        (3)
        )
    anam = (
        'Under 26',
        '26-50',
        '51-75',
        'Over 75'
    )
    distribution = "weibull"
    na = len(aci) # number of age groups considered
    ld = len(census_covariates)
    dextra = 50
    ldd = ld + dextra
    dayrange = range(1,ldd+1)
    output = np.zeros((na,nboot,nstates,ldd))
    counts = np.zeros((nboot,nstates,ldd))
    for jj in tqdm(range(0,na)):
        census = census_covariates[:,jj*(nstates+1):(jj+1)*(nstates+1)]
        counts = np.zeros((nboot,nstates,ldd))
        print("Ensure that growth rates and initial conditions are updated for your local admission trends")
        if (scenario == 1): 
            vect = np.array(range(1,dextra+1)) # vector counting number of days of exponential growth
            if (jj==0): 
                I0 = I0_0 # current expected number of admissions
                rate = rate_0 # growth rate in admissions
                aa = I0*np.exp((rate)*vect) # construct vector of expected number of admissions per day
            elif (jj==1):
                I0 = I0_1 # current expected number of admissions
                rate = rate_1 # growth rate in admissions
                aa = I0*np.exp((rate)*vect) # construct vector of expected number of admissions per day
            elif (jj==2):
                I0 = I0_2 # current expected number of admissions
                rate = rate_2 # growth rate in admissions
                aa = I0*np.exp((rate)*vect) # construct vector of expected number of admissions per day
            elif (jj==3):
                I0 = I0_3 # current expected number of admissions
                rate = rate_3 # growth rate in admissions
                aa = I0*np.exp((rate)*vect) # construct vector of expected number of admissions per day
                
            if approx_growth: # approximate the growth rate using the growth rate across all ages if necessary
                rate = rate_overall
                aa = I0*np.exp((rate)*vect) # construct vector of expected number of admissions per day

        else:
            if (jj==0):
                aa = I0_0 # number of 0-25 admissions per day
                aa = np.mean(census[-10:,5])
            elif (jj==1):
                aa = I0_1 # number of 26-50 admissions per day
                aa = np.mean(census[-10:,5])
            elif (jj==2):
                aa = I0_2 # number of 51-75 admissions per day
                aa = np.mean(census[-10:,5])
            elif (jj==3):
                aa = I0_3 # number of 76+ admissions per day
                aa = np.mean(census[-10:,5])
 
        index_other = aci[jj]+8
        index_shape = aci[jj]+4
        index_scale = aci[jj]
        for s in tqdm(range(0,nboot)):
            ind = random.randint(0,chain_num*draw_num-1)
            kmatb = np.zeros((nstates,nstates)) 
            scalematb = np.zeros((nstates,nstates))
            othermatb = np.zeros((nstates,nstates))
            for k in range(0,ntrans):
                if not global_dist: 
                    if (k==3 or k==4 or k==5):# If we don't assume a global distribution, fit combination of distributions
                        distribution = "weibull" # Fit ICU to death and ICU to recovery using Weibull distributions
                    else:
                        distribution = "burr" # Fit remaining transitions using Burr distributions
                if (distribution == "weibull"):
                    ij = np.argwhere(TransNum == (k+1))[0]
                    kmatb[ij[0],ij[1]] = (xhats[k,index_shape,ind])
                    scalematb[ij[0],ij[1]] = get_scale(xhats[k,index_scale,ind])
                    othermatb[ij[0],ij[1]] = float('nan')

                if (distribution == "burr"):
                    ij = np.argwhere(TransNum == (k+1))[0]
                    kmatb[ij[0],ij[1]] = (xhats[k,index_shape,ind])
                    scalematb[ij[0],ij[1]] = (xhats[k,index_scale,ind])
                    othermatb[ij[0],ij[1]] = (xhats[k,index_other,ind])

                if not (distribution == "burr" or distribution == "weibull"):
                    print("Not fitting correct distribution - please use Weibull or Burr")
                    exit()




            adm = np.zeros(ldd) 
            adm[0:len(census[:,5])] = census[:,5] # admissions
            
            ### Add noise to admissions data by assuming drawn from a poisson distribution about the expected value
            if (scenario ==0):
                adm[len(census[:,5]):] = np.random.poisson(lam=aa,size=dextra)
            else:
                adm[len(census[:,5]):] = np.random.poisson(lam=aa)
            ###

            for d in dayrange:      
                for i in range(0,int(adm[d-1])):
                    y=0
                    Yi = np.array([y])
                    Ti = np.array([0.0])
                    while ((np.sum(Qbase[y,:]) > 0.0) and (Ti[-1]<=ldd)):
                        zz = np.where(Qbase[y,:] > 0.0)[0]
                        tti = np.zeros(len(zz))
                        yyi = np.zeros(len(zz))
                        for q, z in enumerate(zz):
                            if np.isnan(othermatb[y,z]): # If transition is given by Weibull
                                k = kmatb[y,z]
                                scaleg = scalematb[y,z]
                                if np.isnan(k):
                                    tti[q] = float('inf') # if transition not observed, set to inf to it can't take place
                                else:
                                    tti[q] = scaleg*np.random.weibull(k)
                                yyi[q] = z
                            else:          # If transition is given by Burr
                                k = kmatb[y,z]
                                scaleg = scalematb[y,z]
                                otherp = othermatb[y,z]
                                if np.isnan(k):
                                    tti[q] = float('inf') # if transition not observed, set to inf to it can't take place
                                else:
                                    tti[q] = st.burr12.rvs(otherp, k, scale = scaleg)
                                yyi[q] = z                                           

                        r = np.argmin(tti)
                        tran = TransNum[y,:][np.nonzero(TransNum[y,:])][r]
                        y = int(yyi[r])
                        Yi = np.append(Yi,y)
                        Ti = np.append(Ti,Ti[-1].copy() + tti[r])
                    Zi = [int(np.round(x)) for x in Yi]
                    Di = [(d + int(np.round(x))) for x in Ti]
                    for m in range(0,len(Di)):
                        j = Zi[m]
                        if (Di[m]>=ldd):
                            break
                        if (m==(len(Di)-1)):
                            counts[s,j,Di[m]:] += 1.0
                        elif (Di[m+1]>ldd):
                            counts[s,j,Di[m]:ldd] += 1.0
                        else:
                            counts[s,j,Di[m]:Di[m+1]] += 1.0

            output[jj,:,:,:] = counts
    #counts = output.sum(axis=0)

    return(output)

In [ ]:
#### Simulate for posterior credibility - treating each set of covariates as independent
def posterior_simulator(npat,nboot):

    #### Define age indicator variable and add the variable group names
    aci = (
        (0),
        (1),
        (2),
        (3)
        )
    anam = (
        'Under 26',
        '26-50',
        '51-75',
        'Over 75'
    )
    distribution = "weibull"
    na = len(aci) # number of age groups considered

    outdict = {x: np.zeros((nboot,na,2)) for x in allowed_transitions}
    states_to_start = ['Acute Ward','ICU', 'Recovery Ward']


    for j in tqdm(range(0,na)):

        kmatb = np.zeros((nstates,nstates))
        scalematb = np.zeros((nstates,nstates))

        for s in tqdm(range(0,nboot)):
            ind = random.randint(0,chain_num*draw_num-1) # random index to sample parameters from posterior distribution
            kmatb = np.zeros((nstates,nstates)) 
            scalematb = np.zeros((nstates,nstates))
            othermatb = np.zeros((nstates,nstates))
            index_other = aci[j]+8
            index_shape = aci[j]+4
            index_scale = aci[j]
            for k in range(0,ntrans):
                
                if not global_dist: 
                    if (k==3 or k==4 or k==5):# If we don't assume a global distribution, fit combination of distributions
                        distribution = "weibull" # Fit ICU to death and ICU to recovery using Weibull distributions
                    else:
                        distribution = "burr" # Fit remaining transitions using Burr distributions
                if (distribution == "weibull"):
                    ij = np.argwhere(TransNum == (k+1))[0]
                    kmatb[ij[0],ij[1]] = (xhats[k,index_shape,ind])
                    scalematb[ij[0],ij[1]] = get_scale(xhats[k,index_scale,ind])
                    othermatb[ij[0],ij[1]] = float('nan')

                if (distribution == "burr"):
                    ij = np.argwhere(TransNum == (k+1))[0]
                    kmatb[ij[0],ij[1]] = (xhats[k,index_shape,ind])
                    scalematb[ij[0],ij[1]] = (xhats[k,index_scale,ind])
                    othermatb[ij[0],ij[1]] = (xhats[k,index_other,ind])

                if not (distribution == "burr" or distribution == "weibull"):
                    print("Not fitting correct distribution - please use Weibull or Burr")
                    exit()

            for i in range(0,npat):
                for ss in states_to_start: # For each patient start in each start state
                    y = state_dict[ss]-1

                    zz = np.where(Qbase[y,:] > 0.0)[0]
                    tti = np.zeros(len(zz))
                    yyi = np.zeros(len(zz))
                    for q, z in enumerate(zz):
                        if np.isnan(othermatb[y,z]): # If transition is given by Weibull
                            k = kmatb[y,z]
                            scaleg = scalematb[y,z]
                            if np.isnan(k):
                                tti[q] = float('inf')
                            else:
                                tti[q] = scaleg*np.random.weibull(k)
                            yyi[q] = z
                        else:          # If transition is given by Burr
                            k = kmatb[y,z]
                            scaleg = scalematb[y,z]
                            otherp = othermatb[y,z]
                            tti[q] = float('inf')
                            if np.isnan(k):
                                tti[q] = float('inf')
                            else:
                                tti[q] = st.burr12.rvs(otherp, k, scale = scaleg)
                            yyi[q] = z   


                    r = np.argmin(tti)

                    se = state_names[int(yyi[r])]
                    outdict[(ss,se)][s,j,0] += tti[r]
                    outdict[(ss,se)][s,j,1] += 1
    return(outdict)

In [ ]:
'Define required functions - end'

In [ ]:
#### Fitting Distributions 
distribution = "weibull" # if using global distribution, set as Weibull or Burr as required


ntrans = np.max(TransNum) # Number of transitions to model
xhats = np.zeros((ntrans, 15, chain_num*draw_num)) # pre-allocate output variable

rsmin = 35 # random seed range. Occasionally first choice may fail to converge, so we can iterate over
rsmax = 40 # multiple choices of random seed. Hopefully not needed. 

for k in tqdm(range(0,ntrans)): # loop over each transition
    if not global_dist: 
        if (k==3 or k==4 or k==5):# If we don't assume a global distribution, fit combination of distributions
            distribution = "weibull" # Fit ICU to death, ICU to recovery, and recovery to discharge  using Weibull distributions
        else:
            distribution = "burr" # Fit remaining transitions using Burr distributions
    
    ##### Find data with the transition of interest
    dfk = df[df.TransitionNumber == (k+1)]
    T = dfk.TimeInState.values
    #####
    
    ##### Remove any zero values (if any - these shouldn't be genuine as there should be a slight delay however small)
    jj = T>0. # If we get issues with the lengths not matching up, might be caused here...
    T = T[jj] # ...If this is the case, amend data to change any zero durations to 0.1 days and try again.
    #####
    
    ##### Find location of right-censored data points
    C = dfk.Observed.values[jj]
    D = dfk.Age.values[jj]    
    #####
    
    ##### Define initial conditions and number of variables to fit
    'x0 is the initial condition. One set of variable for each set of covariates'
    'X is a vector identifying the location of individuals with each set of covariates'
    'This is used to determine which variables are fitted to a given individual'
    'x is a length indicator for storing the output samples'
    if (distribution=="weibull"): # If using Weibull
        X = np.stack([ (1*(D == 0.0)), (1*(D == 1.0)), 
                      (1*(D == 2.0)), (1*(D == 3.0))]) 
        x = 4
        
    if (distribution=="burr"): # If using Burr
        X = np.stack([ (1*(D == 0.0)), (1*(D == 1.0)), 
                      (1*(D == 2.0)), (1*(D == 3.0))])
        x = 4
        
    if not (distribution == "burr" or distribution == "weibull"):
        print("Not fitting correct distribution - please use Weibull or Burr")
        exit()
    # True / false for censoring (note ~censored captures the full observations)
    censored = C == 0
       
    with pm.Model() as weib_aft_model:        
        def weibull_lsf(x, kk, la):
            'log survival function for Weibull distribution'
            return -(x / la)**kk

        def likelihood1(value,kkinput,lainput,ccinput):
            'log likelihood for Burr distribution'
            c = (ccinput)
            k = (kkinput)
            l = (lainput)
            v1 = c*k/l
            v2 = (value/l)**(c-1)
            v3 = 1 + (value/l)**c
            return pm.math.log(v1*v2*v3**(-k-1)) 

        def likelihood2(value,kkinput,lainput,ccinput):
            'log survival function for Burr distribution'
            c = (ccinput)
            k = (kkinput)
            l = (lainput)
            v1 = 1 + (value/l)**c
            return -k*pm.math.log(v1)
        
        if (distribution=="weibull"):
            ##### Scale parameter comes from a linear combination of parameters with normal priors
            beta = pm.Normal('beta', mu=0., sigma=5., shape=x)
            la = np.exp(-beta.dot(X))
            #####
            
            ##### Shape parameter prior of exponential
            kk = pm.Exponential('kk', lam=1, shape = x)
            la2 = kk.dot(X)
            #####
            
            ##### likelihood for censored observations
            if len(C)>0:
                y_cens = pm.Potential('y_cens', weibull_lsf(T[censored],la2[censored], la[censored]))
            #####
            
            ##### likelihood for observed observations
            y_obs = pm.Weibull('y_obs', alpha=la2[~censored], beta=la[~censored], observed=T[~censored])
            #####

        if (distribution == "burr"):
            ##### Exponential prior for all parameters
            la = pm.Exponential('la', lam=0.2, shape = x)
            kk = pm.Exponential('kk', lam=0.2, shape = x)
            cc = pm.Exponential('cc', lam=0.2, shape = x)
            la2 = la.dot(X)
            kk2 = kk.dot(X)
            cc2 = cc.dot(X)
            #####
            
            ##### likelihood for censored observations
            if len(C)>0:
                y_cens = pm.Potential('y_cens', likelihood2(T[censored],kk2[censored], la2[censored], cc2[censored]))           
            #####
            
            ##### likelihood for observed observations
            y_obs = pm.Potential('y_obs', likelihood1(T[~censored],kk2[~censored], la2[~censored], cc2[~censored]))
            #####

    # This sometimes fails with a bad initial energy - resolved through random seed?
    # Try resolving through a try and catch.
    
    rs = rsmin # first random seed attempt
    
    mcmc_run_ok = False
    
    while (not mcmc_run_ok) and (rs<rsmax): # if there hasn't been a successful MCMC run, repeat for different random seed
        try:
            with weib_aft_model:
                # Tune and change init to avoid divergences
                if saved: # loading saved traces
                    trace = pm.load_trace('multistate_trace'+file_date+'_'+str(k+1)+'.trace')
                else: # running a new sample
                    trace = pm.sample(
                        draws=draw_num,
                        tune=tune_num,
                        target_accept=0.9,
                        init='adapt_diag',
                        random_seed=rs,
                        cores = min((core_num, chain_num)), # numbers of cores needs to be less than or equal to chain_num
                        progressbar=True,
                        chains = chain_num
                    )
            mcmc_run_ok = True # after succesful MCMC run change indicator
        except:
            rs += 1 # move to new random seed if MCMC fails
    if (rs < rsmax): # Store output trace into xhats array
        if (distribution=="weibull"):
            xhats[k,0:x,:] = np.transpose(trace.get_values('beta',burn = 0)) # Weibull
            xhats[k,x:2*x,:] = np.transpose(trace.get_values('kk',burn = 0)) # Weibull
        if (distribution=="burr"):
            xhats[k,0:x,:] = np.transpose(trace.get_values('la',burn = 0)) # Burr
            xhats[k,x:2*x,:] = np.transpose(trace.get_values('kk',burn = 0)) # Burr
            xhats[k,2*x:3*x,:] = np.transpose(trace.get_values('cc',burn = 0)) # Burr
        if not saved:
            pm.save_trace(trace,'multistate_trace'+file_date+'_'+str(k+1)+'.trace',overwrite = True)
    else:
        print("MCMC failed for transition " + str(k+1))

In [ ]:
### For transitions that are not observed in the data, set the first parameter to nan. This will cause the simulator to
### ignore these transitions
print("Ensure that you update which age groups go to ICU")


#### For age groups without ICU stays, set ICU transition parameters to NaN
if not ICU_0:
    xhats[0,0,:] = float('nan')
    xhats[3,0,:] = float('nan')
    xhats[4,0,:] = float('nan')
    xhats[5,0,:] = float('nan')
    xhats[0,4,:] = float('nan')
    xhats[3,4,:] = float('nan')
    xhats[4,4,:] = float('nan')
    xhats[5,4,:] = float('nan')
if not ICU_1:
    xhats[0,1,:] = float('nan')
    xhats[3,1,:] = float('nan')
    xhats[4,1,:] = float('nan')
    xhats[5,1,:] = float('nan')
    xhats[0,5,:] = float('nan')
    xhats[3,5,:] = float('nan')
    xhats[4,5,:] = float('nan')
    xhats[5,5,:] = float('nan')
if not ICU_2:
    xhats[0,2,:] = float('nan')
    xhats[3,2,:] = float('nan')
    xhats[4,2,:] = float('nan')
    xhats[5,2,:] = float('nan')
    xhats[0,6,:] = float('nan')
    xhats[3,6,:] = float('nan')
    xhats[4,6,:] = float('nan')
    xhats[5,6,:] = float('nan')
if not ICU_3:
    xhats[0,3,:] = float('nan')
    xhats[3,3,:] = float('nan')
    xhats[4,3,:] = float('nan')
    xhats[5,3,:] = float('nan')
    xhats[0,7,:] = float('nan')
    xhats[3,7,:] = float('nan')
    xhats[4,7,:] = float('nan')
    xhats[5,7,:] = float('nan')

In [ ]:
'Model fitting - end'

In [ ]:
'Simulations and model output - start'

In [ ]:
#### Call the predictions simulator based on two scenarios, exponential growth and constant admissions
# Admission rates and numbers need to be added to the function
counts_growth_sum = indicative_simulator(1,nboot)
counts_constant_sum = indicative_simulator(0,nboot)

In [ ]:
counts_growth = counts_growth_sum.sum(axis=0)
counts_constant = counts_constant_sum.sum(axis=0)

discharge_counts = counts_growth[:,3,:] - np.concatenate([np.zeros((nboot,1)),counts_growth[:,3,:-1]],axis = 1)
death_counts = counts_growth[:,4,:] - np.concatenate([np.zeros((nboot,1)),counts_growth[:,4,:-1]],axis = 1)

discharge_counts_low = counts_constant[:,3,:] - np.concatenate([np.zeros((nboot,1)),counts_constant[:,3,:-1]],axis = 1)
death_counts_low = counts_constant[:,4,:] - np.concatenate([np.zeros((nboot,1)),counts_constant[:,4,:-1]],axis = 1)

In [ ]:
### Sort probabilities of bed occupancies. Generate vector of bed occupancy over time, broken down into weeks. 

output_acute_med = {}
output_acute_high = {}
output_icu_med = {}
output_icu_high = {}
output_rec_med = {}
output_rec_high = {}
output_discharge_med = {}
output_discharge_high = {}
output_death_med = {}
output_death_high = {}
output_acute_low = {}
output_icu_low = {}
output_rec_low = {}
output_discharge_low = {}
output_death_low = {}


max_len = int((counts_growth.shape[2]-len(census_covariates[:,5]))/7)
for x in range(0,max_len):
    index = (forecast_start - cur_date).days + 7*(x) + len(census_covariates[:,5])
    occupancy_acute = counts_growth[:,0,index]
    occupancy_icu = counts_growth[:,1,index]
    occupancy_rec = counts_growth[:,2,index]
    occupancy_discharge = discharge_counts[:,index]
    occupancy_death = death_counts[:,index]
    occupancy_acute_low = counts_constant[:,0,index]
    occupancy_icu_low = counts_constant[:,1,index]
    occupancy_rec_low = counts_constant[:,2,index]
    occupancy_discharge_low = discharge_counts_low[:,index]
    occupancy_death_low = death_counts_low[:,index]
    output_acute_med[x] = np.nanpercentile(occupancy_acute,50)
    output_acute_high[x] = np.nanpercentile(occupancy_acute,95)
    output_icu_med[x] = np.nanpercentile(occupancy_icu,50)
    output_icu_high[x] = np.nanpercentile(occupancy_icu,95)
    output_rec_med[x] = np.nanpercentile(occupancy_rec,50)
    output_rec_high[x] = np.nanpercentile(occupancy_rec,95)
    output_discharge_med[x] = np.nanpercentile(occupancy_discharge,50)
    output_discharge_high[x] = np.nanpercentile(occupancy_discharge,95)
    output_death_med[x] = np.nanpercentile(occupancy_death,50)
    output_death_high[x] = np.nanpercentile(occupancy_death,95)
    output_acute_low[x] = np.nanpercentile(occupancy_acute_low,50)
    output_icu_low[x] = np.nanpercentile(occupancy_icu_low,50)
    output_rec_low[x] = np.nanpercentile(occupancy_rec_low,50)
    output_discharge_low[x] = np.nanpercentile(occupancy_discharge_low,50)
    output_death_low[x] = np.nanpercentile(occupancy_death_low,50)


In [ ]:
### Compile bed occupancy predictions into a table. Add risk indicator for each bed planning scenario.


#### Generate output table for acute beds
df_current = [] # preallocate data frame
for x in range(0,max_len):
    status = []
    vect = [output_acute_high[x], output_acute_med[x], output_acute_low[x]]
    new_row = [forecast_start +timedelta(x*7), output_acute_high[x], output_acute_med[x], output_acute_low[x]]
    df_current.append(new_row)
    
### Compile table
df_output = pd.DataFrame(df_current)
df_output.columns = ["Week", "Exponential-case 95%", "Exponential-case 50%", "Constant-case"]
df_output.to_csv('Output_Acute_'+file_date+'.csv', index = False)
###

####


#### Generate output table for ICU beds
df_current = [] # preallocate data frame
for x in range(0,max_len):
    status = []
    vect = [output_icu_high[x], output_icu_med[x], output_icu_low[x]]
    new_row = [forecast_start +timedelta(x*7), output_icu_high[x], output_icu_med[x], output_icu_low[x]]    
    df_current.append(new_row)
    
### Compile table
df_output = pd.DataFrame(df_current)
df_output.columns = ["Week", "Exponential-case 95%", "Exponential-case 50%", "Constant-case"]
df_output.to_csv('Output_ICU_'+file_date+'.csv', index = False)
###

####

#### Generate output table for discharges
df_current = [] # preallocate data frame
for x in range(0,max_len):
    status = []
    vect = [output_discharge_high[x], output_discharge_med[x], output_discharge_low[x]]
    new_row = [forecast_start +timedelta(x*7), output_discharge_high[x], output_discharge_med[x], output_discharge_low[x]]    
    df_current.append(new_row)
    
### Compile table
df_output = pd.DataFrame(df_current)
df_output.columns = ["Week", "Exponential-case 95%", "Exponential-case 50%", "Constant-case"]
df_output.to_csv('Output_discharge_'+file_date+'.csv', index = False)
###


#### Generate output table for deaths
df_current = [] # preallocate data frame
for x in range(0,max_len):
    status = []
    vect = [output_death_high[x], output_death_med[x], output_death_low[x]]
    new_row = [forecast_start +timedelta(x*7), output_death_high[x], output_death_med[x], output_death_low[x]]    
    df_current.append(new_row)
    
### Compile table
df_output = pd.DataFrame(df_current)
df_output.columns = ["Week", "Exponential-case 95%", "Exponential-case 50%", "Constant-case"]
df_output.to_csv('Output_death_'+file_date+'.csv', index = False)
###



In [ ]:
'Simulations and model output - end'

In [ ]:
'Generate figures - start'

In [ ]:
#### Combine age level census into total occupancy
#### Define age indicator variable and add the variable group names
aci = (
    (0),
    (1),
    (2),
    (3)
    )
anam = (
    'Under 26',
    '26-50',
    '51-75',
    'Over 75'
)

na = len(aci)

census = np.zeros((len(census_covariates), nstates + 1))

for jj in tqdm(range(0,na)):
    census = census + census_covariates[:,jj*(nstates+1):(jj+1)*(nstates+1)]
    


In [ ]:
#### Generate axis labels for plotting
ld = len(census_covariates)
dextra = 50
ldd = ld + dextra
dayrange = range(1,ldd+1)
dt0 = datetime.fromisoformat('2020-03-01 00:00:01')
dstart = 0 
dd = np.arange(0,ldd,35)
#dd = dayrange
xl = []
for j in range(0,len(dd)):
    xl.append((dt0 + timedelta(days=(dstart+int(dd[j])))).strftime('%d/%m'))

In [ ]:
#### Plot forecasting figures

#### PLot growth scenario
plt.figure(figsize=(8,5))

for i in range(0,3):
    
    plt.subplot(2,2,i+1)
    for s in range(0,nboot):
        plt.plot(dayrange,counts_growth[s,i,:],c='k',linewidth=2,alpha=0.1,zorder=0)  
    plt.title(state_names[i])
    plt.xlabel('Day')
    plt.ylabel('Bed Occupancy')
    plt.xticks(dd,xl)
    plt.xticks(rotation=90)


plt.subplot(2,2,4)
for s in range(0,nboot):
    plt.plot(dayrange,np.sum(counts_growth[s,[3,4],:],0),c='k',linewidth=2,alpha=0.1,zorder=0)  
plt.title('Outcome')
plt.xlabel('Day')
plt.ylabel('Count')
plt.xticks(dd,xl)
plt.xticks(rotation=90)

plt.tight_layout()
plt.savefig('indicative_growth_'+file_date+'.pdf')

#### Plot constant scenario
plt.figure(figsize=(8,5))

for i in range(0,3):
    
    plt.subplot(2,2,i+1)
    for s in range(0,nboot):
        plt.plot(dayrange,counts_constant[s,i,:],c='k',linewidth=2,alpha=0.01,zorder=0)  
    plt.title(state_names[i])
    plt.xlabel('Day')
    plt.ylabel('Bed Occupancy')
    plt.xticks(dd,xl)
    plt.xticks(rotation=90)


plt.subplot(2,2,4)
for s in range(0,nboot):
    plt.plot(dayrange,np.sum(counts_constant[s,[3,4],:],0),c='k',linewidth=2,alpha=0.01,zorder=0)  
plt.title('Outcome')
plt.xlabel('Day')
plt.ylabel('Count')
plt.xticks(dd,xl)
plt.xticks(rotation=90)

plt.tight_layout()
plt.savefig('indicative_stable_'+file_date+'.pdf')

In [ ]:
#### Plot for comparison with census
dayrange = range(0,ldd) #

plt.figure(figsize=(6,5))

for i in range(0,3):
    
    plt.subplot(2,2,i+1)
    for s in range(0,nboot):
        plt.plot(dayrange,counts_growth[s,i,:],c='k',linewidth=2,alpha=0.1,zorder=0) 
    z = census[:,i]
    w = np.array([])
    for day in range(0,len(z)):
        w = np.append(w,np.repeat(day, int(z[day])))
    t = range(0, len(z))
    plt.plot(t, (z), c='w', lw=2, zorder=2)
    plt.plot(t, (z), c='r', lw=1, zorder=3)
    plt.title(state_names[i])
    plt.xlim([0,len(census)+20])
    plt.xlabel('Day')
    plt.ylabel('Bed Occupancy')
    plt.xticks(dd,xl)
    plt.xticks(rotation=90)
plt.subplot(2,2,4)
for s in range(0,nboot):
    plt.plot(dayrange,np.sum(counts_growth[s,[3,4],:],0),c='k',linewidth=2,alpha=0.1,zorder=0)  
z = np.sum(census[:,[3,4]],1)
w = np.array([])
for day in range(0,len(z)):
    w = np.append(w,np.repeat(day, int(z[day])))
kde = st.gaussian_kde(w)
plt.plot(t, z, c='w', lw=2, zorder=2)
plt.plot(t, z, c='r', lw=1, zorder=3)
plt.title('Outcome')
plt.xlim([0,len(census)+20])
plt.xlabel('Day')
plt.ylabel('Count')
plt.xticks(dd,xl)
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig('modelfit_'+file_date+'.pdf')

In [ ]:
'Generate figures - end'

In [ ]:
'Generate length of stay estimates - start'

In [ ]:
#### Call the posterior simulator to give mean and uncertainty estimates for the observed lengths of stay
if (generate_los_estimates == 1):

    outdict_sum = posterior_simulator(npat,nboot)

    #### Work out probabilities of going to each state and display length of stay and probabilities plus uncertainty

    #### Define age indicator variable and add the variable group names
    aci = (
        (0),
        (1),
        (2),
        (3)
        )
    anam = (
        'Under 26',
        '26-50',
        '51-75',
        'Over 75'
    )

    na = len(aci) # number of age groups considered
    states_to_start = ['Acute Ward','ICU', 'Recovery Ward']

    for j in range(0,na):

        ss = states_to_start[0]
        pvecb = []
        prowb  = []
        for pair, vec in outdict_sum.items():
            if (pair[0] == ss):
                prowb.append(outdict_sum[pair][:,j,1])
            else:
                ss = pair[0]
                pvecb.append(prowb)
                prowb = []
                prowb.append(outdict_sum[pair][:,j,1])
        pvecb.append(prowb)

        probs = np.array([])
        pl = np.array([])
        pu = np.array([])
        for u in range(0,len(pvecb)):
            qa = np.array(pvecb[u])*(100/npat)
            qvec = np.nanpercentile(qa,50,1)
            ql = np.nanpercentile(qa,2.5,1)
            qu = np.nanpercentile(qa,97.5,1)
    #         ql = np.nanpercentile(qa,5,1)
    #         qu = np.nanpercentile(qa,95,1)
            probs = np.concatenate([probs,qvec])
            pl = np.concatenate([pl,ql])
            pu = np.concatenate([pu,qu])


        print('*** {} ***'.format(anam[j]))
        u = 0
        for pair, vec in outdict_sum.items():
            tts = outdict_sum[pair][:,j,0]/outdict_sum[pair][:,j,1]
            that = np.nanpercentile(tts,50)
            tlow = np.nanpercentile(tts,2.5)
            tup = np.nanpercentile(tts,97.5)
            print('{} to {}: {:.1f}({:.1f},{:.1f}) days; probability {:.1f}({:.1f},{:.1f})%'.format(
                pair[0], pair[1], that, tlow, tup, probs[u], pl[u], pu[u]))
            u += 1
        print('')

In [ ]:
'Generate length of stay estimates - end'